In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from pathlib import Path

# pd.set_option('display.max_columns', 200)

import allensdk.brain_observatory.behavior.behavior_project_cache as bpc

import allensdk
import pkg_resources
print('allensdk version 2.10.2 or higher is required, you have {} installed'.format(pkg_resources.get_distribution("allensdk").version))
import mindscope_utilities.visual_behavior_ophys as ophys
import mindscope_utilities

C:\Users\sepeh\.conda\envs\allen-sdk\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\sepeh\.conda\envs\allen-sdk\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\sepeh\.conda\envs\allen-sdk\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\sepeh\.conda\envs\allen-sdk\lib\site-packages\numpy\.libs\libopenblas.xwydx2ikjw2nmtwsfyngfuwkqu3lytcz.gfortran-win_amd64.dll
  stacklevel=1)


allensdk version 2.10.2 or higher is required, you have 2.13.2 installed


In [2]:
# %matplotlib notebook
# %matplotlib inline
# %matplotlib qt 
%matplotlib widget

In [3]:
import visual_behavior.data_access.utilities as utilities

C:\Users\sepeh\AppData\Roaming\Python\Python37\site-packages\visual_behavior\data_access\from_lims.py:37: UserWarning: failed to set up LIMS/mtrain credentials
'LIMS_DBNAME'

         internal AIBS users should set up environment variables         appropriately
functions requiring database access will fail
  warnings.warn(warn_string)
C:\Users\sepeh\AppData\Roaming\Python\Python37\site-packages\visual_behavior\data_access\loading.py:53: UserWarning: failed to set up LIMS/mtrain credentials
'LIMS_DBNAME'

internal AIBS users should set up environment variables appropriately
functions requiring database access will fail
  warnings.warn(warn_string)


In [35]:
my_cache_dir = Path("G:/allensdk")

bc = bpc.VisualBehaviorOphysProjectCache.from_s3_cache(cache_dir=my_cache_dir)
          
experiments_table = bc.get_ophys_experiment_table()  
cell_table = bc.get_ophys_cells_table()

In [5]:
experiments_table.cre_line.unique()

array(['Sst-IRES-Cre', 'Vip-IRES-Cre', 'Slc17a7-IRES2-Cre'], dtype=object)

In [10]:
experiments_table = add_cell_type_column(experiments_table)

In [11]:
all_expts = experiments_table.copy()
limit_expts = experiments_table.copy()

In [12]:
all_expts = experiments_table[experiments_table.passive==False]
limit_expts = experiments_table[experiments_table.passive==False]
limit_expts = utilities.limit_to_containers_with_all_experience_levels(limit_expts)

### distribution of prior exposures to image set

In [13]:
all_expts.groupby(['cell_type', 'experience_level', 'prior_exposures_to_image_set']).mean().reset_index().groupby(['cell_type', 'experience_level']).describe()[['prior_exposures_to_image_set']]


prior_exposures_to_image_set             \
                                                       count       mean   
cell_type      experience_level                                           
Excitatory     Familiar                                 45.0  34.044444   
               Novel 1                                   1.0   0.000000   
               Novel >1                                 10.0   5.500000   
Sst Inhibitory Familiar                                 38.0  34.763158   
               Novel 1                                   1.0   0.000000   
               Novel >1                                  6.0   3.500000   
Vip Inhibitory Familiar                                 36.0  31.861111   
               Novel 1                                   1.0   0.000000   
               Novel >1                                  7.0   4.000000   

                                                                            
                                       std   min    25%   50%    75%   max  
cell_type      experience_level                                             
Excitatory     Familiar          18.432870   8.0  19.00  31.0  53.00  68.0  
               Novel 1                 NaN   0.0   0.00   0.0   0.00   0.0  
               Novel >1           3.027650   1.0   3.25   5.5   7.75  10.0  
Sst Inhibitory Familiar          18.474869   9.0  20.25  31.5  42.75  74.0  
               Novel 1                 NaN   0.0   0.00   0.0   0.00   0.0  
               Novel >1           1.870829   1.0   2.25   3.5   4.75   6.0  
Vip Inhibitory Familiar          15.214941  10.0  18.75  27.5  46.25  57.0  
               Novel 1                 NaN   0.0   0.00   0.0   0.00   0.0  
               Novel >1           2.160247   1.0   2.50   4.0   5.50   7.0

In [ ]:
tmp = experiments_table[experiments_table.passive==False].copy()
# len(tmp)
tmp.groupby(['cell_type', 'experience_level', 'prior_exposures_to_image_set']).mean().reset_index().groupby(['cell_type', 'experience_level']).apply(display)


In [15]:
# tmp = experiments_table[experiments_table.passive==False].copy()
# tmp.groupby(['cell_type', 'experience_level', 'prior_exposures_to_image_set']).mean()

In [16]:
# experiments_table[(experiments_table.experience_level == 'Novel 1') & (experiments_table.prior_exposures_to_image_set == 1)]

this shows Novel 1 sessions only include 0 exposures to image set

In [17]:

tmp = experiments_table[experiments_table.passive==False].copy()
# tmp.set_index(['cell_type', 'experience_level'], inplace=True)
distrib_df = tmp.groupby(['cell_type', 'experience_level']).count()[['session_number']]
distrib_df

session_number
cell_type      experience_level                
Excitatory     Familiar                     309
               Novel 1                      102
               Novel >1                     202
Sst Inhibitory Familiar                     154
               Novel 1                       58
               Novel >1                      80
Vip Inhibitory Familiar                     230
               Novel 1                       75
               Novel >1                     162

In [18]:
import dtale
dtale.show(distrib_df)

In [19]:
merge_cell_exp = cell_table.copy()
merge_cell_exp = merge_cell_exp.reset_index().merge(experiments_table, on='ophys_experiment_id')
merge_cell_exp = merge_cell_exp.set_index('cell_roi_id')
merge_cell_exp

,cell_specimen_id,ophys_experiment_id,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,cre_line,...,project_code,imaging_depth,targeted_structure,date_of_acquisition,session_type,experience_level,passive,image_set,file_id,cell_type
cell_roi_id,,,,,,,,,,,,,,,,,,,,,
1080884343,1086496928,775614751,CAM2P.5,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,403491,Ai93(TITL-GCaMP6f),"[Slc17a7-IRES2-Cre, Camk2a-tTA]",F,160.0,Slc17a7-IRES2-Cre,...,VisualBehavior,375,VISp,2018-11-08 18:38:05.000000,OPHYS_1_images_A,Familiar,False,A,945253901,Excitatory
1080884173,1086496914,775614751,CAM2P.5,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,403491,Ai93(TITL-GCaMP6f),"[Slc17a7-IRES2-Cre, Camk2a-tTA]",F,160.0,Slc17a7-IRES2-Cre,...,VisualBehavior,375,VISp,2018-11-08 18:38:05.000000,OPHYS_1_images_A,Familiar,False,A,945253901,Excitatory
1080883843,1086496838,775614751,CAM2P.5,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,403491,Ai93(TITL-GCaMP6f),"[Slc17a7-IRES2-Cre, Camk2a-tTA]",F,160.0,Slc17a7-IRES2-Cre,...,VisualBehavior,375,VISp,2018-11-08 18:38:05.000000,OPHYS_1_images_A,Familiar,False,A,945253901,Excitatory
1080886674,1086491756,775614751,CAM2P.5,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,403491,Ai93(TITL-GCaMP6f),"[Slc17a7-IRES2-Cre, Camk2a-tTA]",F,160.0,Slc17a7-IRES2-Cre,...,VisualBehavior,375,VISp,2018-11-08 18:38:05.000000,OPHYS_1_images_A,Familiar,False,A,945253901,Excitatory
1080885658,1086491699,775614751,CAM2P.5,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai93(TITL-G...,403491,Ai93(TITL-GCaMP6f),"[Slc17a7-IRES2-Cre, Camk2a-tTA]",F,160.0,Slc17a7-IRES2-Cre,...,VisualBehavior,375,VISp,2018-11-08 18:38:05.000000,OPHYS_1_images_A,Familiar,False,A,945253901,Excitatory
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115385589,1120127229,1110928073,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,570949,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,144.0,Sst-IRES-Cre,...,VisualBehaviorMultiscope4areasx2d,105,VISam,2021-06-22 08:22:01.643464,OPHYS_6_images_H,Novel >1,False,H,1120151758,Sst Inhibitory
1115385513,1120127083,1110928073,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,570949,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,144.0,Sst-IRES-Cre,...,VisualBehaviorMultiscope4areasx2d,105,VISam,2021-06-22 08:22:01.643464,OPHYS_6_images_H,Novel >1,False,H,1120151758,Sst Inhibitory
1115301580,1120126794,1110928075,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,570949,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,144.0,Sst-IRES-Cre,...,VisualBehaviorMultiscope4areasx2d,167,VISam,2021-06-22 08:22:01.643464,OPHYS_6_images_H,Novel >1,False,H,1120151751,Sst Inhibitory


In [20]:
cells = merge_cell_exp.copy()
cells = cells[cells.ophys_experiment_id.isin(all_expts.index.values)]
tmp1 = utilities.count_mice_expts_containers_cells(cells)

In [21]:
dtale.show(tmp1)

## find mice with all crelines??

In [39]:
mouse_id_list_all_exp = []
for mouse_id in experiments_table.mouse_id:
    tmp_sel = experiments_table[experiments_table.mouse_id == mouse_id]
    # print(tmp_sel.cre_line.nunique())
    if tmp_sel.cre_line.nunique() != 1:
        mouse_id_list_all_exp.append(mouse_id)

In [40]:
mouse_id_list_all_exp

[]

#### mice only have one creline 

In [ ]:
table = experiments_table[(experiments_table.image_set == 'A') | (experiments_table.image_set == 'B')]
len(table)

In [50]:
limit_expts = utilities.limit_to_containers_with_all_experience_levels(experiments_table)

In [51]:
dtale.show(limit_expts)

Executing shutdown...
Executing shutdown...


2022-05-06 16:57:03,333 - INFO     - Executing shutdown...


In [38]:
experiments_table

,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,cre_line,indicator,session_number,...,ophys_container_id,project_code,imaging_depth,targeted_structure,date_of_acquisition,session_type,experience_level,passive,image_set,file_id
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,
951980486,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206.0,Sst-IRES-Cre,GCaMP6f,1.0,...,1018028360,VisualBehaviorMultiscope,300,VISl,2019-09-20 09:45:29.897856,OPHYS_1_images_A,Familiar,False,A,1085400869
951980473,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206.0,Sst-IRES-Cre,GCaMP6f,1.0,...,1018028345,VisualBehaviorMultiscope,225,VISp,2019-09-20 09:45:29.897856,OPHYS_1_images_A,Familiar,False,A,1085400919
951980481,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206.0,Sst-IRES-Cre,GCaMP6f,1.0,...,1018028357,VisualBehaviorMultiscope,225,VISl,2019-09-20 09:45:29.897856,OPHYS_1_images_A,Familiar,False,A,1085400972
951980471,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206.0,Sst-IRES-Cre,GCaMP6f,1.0,...,1018028342,VisualBehaviorMultiscope,150,VISp,2019-09-20 09:45:29.897856,OPHYS_1_images_A,Familiar,False,A,1085394308
951980475,MESO.1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,206.0,Sst-IRES-Cre,GCaMP6f,1.0,...,1018028339,VisualBehaviorMultiscope,75,VISp,2019-09-20 09:45:29.897856,OPHYS_1_images_A,Familiar,False,A,1085400920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
947716576,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,468866,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,152.0,Vip-IRES-Cre,GCaMP6f,3.0,...,929913236,VisualBehavior,175,VISp,2019-09-16 16:46:16.000000,OPHYS_3_images_A,Familiar,False,A,947805783
948507789,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,468866,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,153.0,Vip-IRES-Cre,GCaMP6f,3.0,...,929913236,VisualBehavior,175,VISp,2019-09-17 17:39:10.000000,OPHYS_3_images_A,Familiar,False,A,949405820
950833329,CAM2P.4,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,468866,Ai148(TIT2L-GC6f-ICL-tTA2),[Vip-IRES-Cre],F,155.0,Vip-IRES-Cre,GCaMP6f,4.0,...,929913236,VisualBehavior,175,VISp,2019-09-19 15:34:06.000000,OPHYS_4_images_B,Novel 1,False,B,951266634


In [22]:
def count_mice_expts_containers_cells(df):
    """
    count the number of mice, experiments, containers, and cells in input dataframe
    input dataframe is typically ophys_cells_table merged with ophys_experiment_table
    """
    mice = value_counts(df, conditions=['cell_type', 'experience_level', 'mouse_id'])
    experiments = value_counts(df, conditions=['cell_type', 'experience_level', 'ophys_experiment_id'])
    containers = value_counts(df, conditions=['cell_type', 'experience_level', 'ophys_container_id'])
    cells = value_counts(df, conditions=['cell_type', 'experience_level', 'cell_specimen_id'])

    counts = mice.merge(experiments, on=['cell_type', 'experience_level'])
    counts = counts.merge(containers, on=['cell_type', 'experience_level'])
    counts = counts.merge(cells, on=['cell_type', 'experience_level'])
    return counts

def value_counts(df, conditions=['cell_type', 'experience_level', 'mouse_id']):
    """
    group by the first conditions and count the last one
    """
    counts = df.groupby(conditions).count().reset_index().groupby(conditions[:-1]).count()
    counts = counts[[conditions[-1]]].rename(columns={conditions[-1]: 'n_' + conditions[-1]})
    return counts

In [23]:
def add_cell_type_column(df):
    """
    adds a column with abbreviated version of cre_line, i.e. Vip, Sst, Exc
    """
    cre_indices = df[df.cre_line == 'Vip-IRES-Cre'].index.values
    df.loc[cre_indices, 'cell_type'] = 'Vip Inhibitory'

    cre_indices = df[df.cre_line == 'Sst-IRES-Cre'].index.values
    df.loc[cre_indices, 'cell_type'] = 'Sst Inhibitory'

    cre_indices = df[df.cre_line == 'Slc17a7-IRES2-Cre'].index.values
    df.loc[cre_indices, 'cell_type'] = 'Excitatory'

    return df

In [ ]:
def get_containers_with_all_experience_levels(experiments_table):
    """
    identifies containers with all 3 experience levels in ['Familiar', 'Novel 1', 'Novel >1']
    returns a list of container_ids
    """
    experience_level_counts = experiments_table.groupby(['ophys_container_id', 'experience_level']).count().reset_index().groupby(['ophys_container_id']).count()[['experience_level']]
    containers_with_all_experience_levels = experience_level_counts[experience_level_counts.experience_level == 3].index.unique()
    return containers_with_all_experience_levels

def limit_to_containers_with_all_experience_levels(experiments_table):
    """
    returns experiment_table limited to containers with all 3 experience levels in ['Familiar', 'Novel 1', 'Novel >1']
    input dataframe is typically ophys_experiment_table but can be any df with columns 'ophys_container_id' and 'experience_level'
    """
    containers_with_all_experience_levels = get_containers_with_all_experience_levels(experiments_table)
    experiments_table = experiments_table[experiments_table.ophys_container_id.isin(containers_with_all_experience_levels)]
    return experiments_table